# FaceFusion on Google Colab
This notebook installs and runs FaceFusion on Colab with a Gradio interface that you can share.\n> Always respect local laws and the FaceFusion license when processing facial data.

## 1. Check the available GPU

In [ ]:
!nvidia-smi

## 2. Configure repository information
Adjust the repo URL / branch / install directory before running if you need a fork.

In [ ]:
#@title Define repo, branch and install paths
PROJECT_ROOT = "/content/facefusion" #@param {type:"string"}
REPO_URL = "https://github.com/facefusion/facefusion.git" #@param {type:"string"}
BRANCH = "master" #@param {type:"string"}

import os
os.environ['FACEFUSION_PROJECT_ROOT'] = PROJECT_ROOT
os.environ['FACEFUSION_REPO_URL'] = REPO_URL
os.environ['FACEFUSION_BRANCH'] = BRANCH

print(f'Project root: {PROJECT_ROOT}')
print(f'Repository : {REPO_URL} (branch {BRANCH})')

## 3. Clone FaceFusion and install dependencies
Run this every time you start a fresh Colab runtime.

In [ ]:
#@title Clone FaceFusion & install requirements
%%bash
set -e
cd /content
PROJECT_ROOT="${FACEFUSION_PROJECT_ROOT}"
REPO_URL="${FACEFUSION_REPO_URL}"
BRANCH="${FACEFUSION_BRANCH}"

echo "[INFO] Cleaning previous checkout..."
rm -rf "${PROJECT_ROOT}"
echo "[INFO] Cloning ${REPO_URL} (branch ${BRANCH})"
git clone --depth 1 --branch "${BRANCH}" "${REPO_URL}" "${PROJECT_ROOT}"
cd "${PROJECT_ROOT}"
python -m pip install --upgrade pip
python -m pip install -r requirements.txt

## 4. Prepare working folders

In [ ]:
#@title Create input / output folders
from pathlib import Path
import os
base = Path(os.environ['FACEFUSION_PROJECT_ROOT'])
folders = ['data/source', 'data/target', 'data/audio', 'outputs', '.temp', '.jobs']
for rel in folders:
    path = base / rel
    path.mkdir(parents=True, exist_ok=True)
    print(f'[OK] {path}')

os.environ['FACEFUSION_SOURCE_DIR'] = str(base / 'data/source')
os.environ['FACEFUSION_TARGET_DIR'] = str(base / 'data/target')
os.environ['FACEFUSION_AUDIO_DIR'] = str(base / 'data/audio')
os.environ['FACEFUSION_OUTPUT_DIR'] = str(base / 'outputs')

**How to provide media**
- Drag & drop files into the Colab *Files* sidebar and move them into the folders above.
- Or mount Google Drive to reuse assets across sessions.

In [ ]:
#@title (Optional) Mount Google Drive
USE_DRIVE = False #@param {type:"boolean"}
MOUNT_POINT = '/content/drive' #@param {type:"string"}
if USE_DRIVE:
    try:
        from google.colab import drive  # type: ignore
        drive.mount(MOUNT_POINT, force_remount=True)
    except Exception as exc:
        print(f'Could not mount Drive: {exc}')
else:
    print('Skipped. Set USE_DRIVE = True when you need it.')

In [ ]:
#@title (Optional) Quick upload helper
AUTO_UPLOAD = False #@param {type:"boolean"}
DESTINATION = 'source' #@param ['source', 'target', 'audio']
if AUTO_UPLOAD:
    try:
        from google.colab import files  # type: ignore
    except Exception:
        files = None
    if not files:
        print('files.upload() is only available inside the Colab UI.')
    else:
        dest = Path(os.environ['FACEFUSION_PROJECT_ROOT']) / f'data/{DESTINATION}'
        uploaded = files.upload()
        for name, data in uploaded.items():
            (dest / name).write_bytes(data)
            print(f'Saved {name} into {dest}')
else:
    print('Toggle AUTO_UPLOAD to True to open the picker.')

## 5. (Recommended) Download the model weights

In [ ]:
#@title Force-download the models
DOWNLOAD_MODELS = False #@param {type:"boolean"}
if DOWNLOAD_MODELS:
    !cd $FACEFUSION_PROJECT_ROOT && python facefusion.py force-download
else:
    print('Skipping downloads. Set DOWNLOAD_MODELS = True when needed.')

## 6. Launch the FaceFusion UI
Run the cell below and keep it active. Colab will display a **Public URL** (gradio.live) that you can share during this session.

In [ ]:
#@title Start the Gradio UI (provides a shareable link)
!cd $FACEFUSION_PROJECT_ROOT && python facefusion.py run --temp-path $FACEFUSION_PROJECT_ROOT/.temp --jobs-path $FACEFUSION_PROJECT_ROOT/.jobs --output-path $FACEFUSION_PROJECT_ROOT/outputs --open-browser --ui-share

## 7. (Optional) Headless example
Reuse the same paths you configured in the UI for automated runs.

In [ ]:
#@title Headless run sample
RUN_HEADLESS = False #@param {type:"boolean"}
SOURCE_PATH = '/content/facefusion/data/source/source.jpg' #@param {type:"string"}
TARGET_PATH = '/content/facefusion/data/target/target.mp4' #@param {type:"string"}
OUTPUT_PATH = '/content/facefusion/outputs/result.mp4' #@param {type:"string"}
PROCESSORS = 'face_swapper' #@param ['face_swapper', 'face_enhancer', 'frame_enhancer', 'background_remover', 'lip_syncer']
if RUN_HEADLESS:
    !cd $FACEFUSION_PROJECT_ROOT && python facefusion.py headless-run -s $SOURCE_PATH -t $TARGET_PATH -o $OUTPUT_PATH --processors $PROCESSORS
else:
    print('Set RUN_HEADLESS = True to execute the sample command.')

## 8. Sharing tips
- Notebook: click **Share** in Colab and grant Viewer/Editor access as needed.
- Running UI: copy the `Public URL` (https://xxxxx.gradio.live) shown after the UI cell starts; the link dies when the runtime stops.
- Stop everything via the square-stop button on the UI cell or `Runtime → Disconnect and delete runtime`.